## [네이버 영화 한 줄 평 수집](https://movie.naver.com/movie/running/current.naver)
### 2022.01.29

- 원하는 데이터가 페이지 소스보기를 하면 존재하지 않는다.
- 마우스 우클릭을 하면 페이지 소스보기만 있는 것이 아닌 프레임 소스보기 존재한다.
- 이럴 경우 해당 데이터를 가지고 있는 페이지는 iframe으로 삽입됬을 확률이 높다.
- 페이지 소스보기를 한 후 iframe으로 검색한다.
- 검색한 iframe의 src 속성에 설정되어 있는 주소를 확인하여 페이지 주소를 파악한다
- 또는 프레임 소스보기를 클릭해서 나온 화면 주소에서 view-source를 떼버린 주소로 요청한다.

In [1]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, fileName) :
    # 데이터를 담을 딕셔너리
    data_dict = {
        '평점' : [],
        '평가글' : [],
        '작성자' : [], 
        '작성시간' : [],
        '공감수' : [],
        '비공감수' : []
    }
    
    
    # 평가글들을 가져온다
    li_list = source.select('body > div > div > div.score_result > ul > li')
    
    for idx, li_tag in enumerate(li_list) :
#         print(li_tag)
        
        
        # 평점
        a1 = li_tag.select_one('div.star_score > em')     
        if a1 == None :
            data_dict['평점'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['평점'].append(data1)
            
        # 평가글
        a1 = li_tag.select_one(f'#_unfold_ment{idx} > a')     
        if a1 == None :
            a2 = li_tag.select_one(f'#_filtered_ment_{idx}')
            if a2 == None :
                data_dict['평가글'].append(np.nan)
            else :
                data2 = a2.text.strip()
                data_dict['평가글'].append(data2)
        else :
            # data-src 속성의 값을 가져온다.
            data1 = a1.attrs['data-src'].strip()
            data_dict['평가글'].append(data1)
            
        # 작성자
        a1 = li_tag.select_one('div.score_reple > dl > dt > em:nth-child(1) > a > span')     
        if a1 == None :
            data_dict['작성자'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['작성자'].append(data1)
            
        # 작성시간
        a1 = li_tag.select_one('div.score_reple > dl > dt > em:nth-child(2)')     
        if a1 == None :
            data_dict['작성시간'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['작성시간'].append(data1)
            
        # 공감수
        a1 = li_tag.select_one('div.btn_area > a._sympathyButton > strong')    
        if a1 == None :
            data_dict['공감수'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['공감수'].append(data1)
            
            
        # 비공감수
        a1 = li_tag.select_one('div.btn_area > a._notSympathyButton > strong')    
        if a1 == None :
            data_dict['비공감수'].append(np.nan)
        else :
            data1 = a1.text.strip()
            data_dict['비공감수'].append(data1)   
            
    # print(data_dict)
    
    # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    # 저장한다.
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [4]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(source) :
    pass

In [5]:
# 페이지 수를 구하는 함수
def getPageCount(code) :
    
    site = f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"
    
    a1 = getSource(site)
    source = bs4.BeautifulSoup(a1)
    
    # 평가 글 수를 가져온다.
    a2 = source.select_one('body > div > div > div.score_total > strong > em')
    data1 = a2.text.strip()
    
    # 쉼표를 제거한다.
    data1 = data1.replace(',', '')
    
    # 정수값으로 변환한다.
    data1 = int(data1)
    
    # 한페이지에 글이 10개 씩이므로 10으로 나눈다.
    data2 = data1 // 10
    
    if data1 % 10 >  0 :
        data2 = data2 + 1
    
    return data2
    

In [8]:
# 영화 코드를 가져오는 함수.
def getMovieCode() :
    # 파일에서 데이터를 읽어온다.
    df1 = pd.read_csv('naver_movie_info.csv')
    # 영화 코드를 가져온다.
    a1 = df1['영화코드'].values
    
    return a1# 영화 코드를 가져오는 함수.


In [7]:
# 페이지의 주소
page = 1
fileName = 'naver_rating_info.csv'
mode = 'test' # 전체 수집시 다른 단어로 해주세요.

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)

# 영화 코드를 가져온다.
code_list = getMovieCode()

# 테스트용 코드 
if mode == 'test' :
    code_list = code_list[:5]

# 영화 코드만큼 반복한다.
for code in code_list :
    
    # 현재 영화의 페이지 수를 구한다.
    pageCount = getPageCount(code)
    
    # 테스용 코드
    if mode == 'test' :
        if pageCount > 5 :
            pageCount = 5

    # 페이지 번호 초기화
    page = 1
    
    for a100 in range(pageCount) :
        # 딜레이
        time.sleep(1)

        # 화면 청소
        clear_output(wait=True)

        print(f'{code} : {a100 + 1} / {pageCount}')

        site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'

        # 서버에 접속해서 응답 결과를 가져온다.
        a1 = getSource(site)
        source = bs4.BeautifulSoup(a1)

        # 현재 페이지의 데이터를 수집한다.
        getData(source, fileName)

        # 페이지 번호를 증가한다.
        page = page + 1

print('수집 완료')

187320 : 5 / 5
수집 완료
